# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E7%84%A6%E9%BB%9E%E6%96%B0%E8%81%9E/"  #焦點新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='../chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
# https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-in-dynamically-loading-webpage/48851166#48851166
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(1)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 路透專訪王毅「中國已掌控疫情」：美國帶頭過度反應　引起不必要的恐慌

▲中國外交部長王毅認為，若疫情出現在其他國家，沒有領導人能夠勝任。（圖／路透）
記者張靖榕／綜合外電報導
中國新冠肺炎（COVID-19）疫情在各國間引起不同程度的恐慌，但外交部長王毅表示，儘管這款病毒所帶來的影響，是對政府的一項嚴峻挑戰，但北京正掌控著疫情，其他國家的反應都過激了。
王毅在德國柏林接受路透社90分鐘的訪問時，再次呼籲美國不要採取過度且沒有必要的防疫措施，這些做法會對兩國貿易往來及觀光旅遊產業影響重大，「這場疫情爆發的確來的突然，這為中國和全世界帶來了一項挑戰。」然而他再次重申，整體疫情「可防可控」，已在掌控之中。
王毅表示北京當局已經採取全面預防及管控措施，「防疫工作做的如此全面，我還沒看過有任何國家可以做到這種程度。」他稱讚中國國家主席習近平的能力，因為若把疫情放到其他國家，任何一個領導人都會覺得這項挑戰太過艱難，「但中國有能力面對這件事」。
這場訪問涉及的議題包括新冠肺炎、香港以及中東問題，記者注意到王毅在每項議題上，都認為美國華盛頓當局需要負起責任。
回顧各國因疫情擴散而紛紛包機撤僑的決定，王毅批評就是美國打響第一槍，撤僑又發出旅遊限制，才會造成後續效應，「部分國家採取的應對方法，包括隔離措施在內，是合理且可以理解的，但有些國家的反應過激，引起了不必要的恐慌。」
王毅對這些「引起非必要恐慌的國家」撂下重話，表示他們最終會逐漸明白疫情已經得到控制，「因為到最後，這些國家還是需要和中國互動。」
----------------------------------------------------------------------

[1] 台灣防疫網「兩大挑戰時間點」曝光　急診醫：能撐到現在已是奇蹟

記者黃翊婷／綜合報導
中國醫藥大學醫學中心兒童急診科主任謝宗學14日才在臉書PO文，呼籲民眾近期最好別去日本、新加坡旅遊。同日晚間他再度PO文表示，台灣的防疫網仍有「兩大時間點」的嚴峻考驗，分別是3月中下旬以及清明返鄉掃墓潮（4月中下旬），周圍國家一個接一個淪陷，「唯獨台灣還未發生社區疫情，然而，我們還能撐多久？」
▲日本光是